# Problem Statement — Tiny Shop Analytics (NumPy + pandas)

You’re given a small synthetic dataset of shop transactions. Each row has:

* `date` – calendar day of the transaction
* `store` – which store made the sale (`"A"` or `"B"`)
* `category` – product category (`"Food"`, `"Home"`, `"Toys"`)
* `price` – base unit price (some values are missing)
* `qty` – quantity purchased (some values are missing)
* `discount` – fraction discount applied (0.0, 0.1, 0.2)

Your tasks:

1. **Handle missing values**

   * Forward-fill `price` within each `category`; if any `price` still missing, fill with that category’s **median price**.
   * Fill missing `qty` with **1**.

2. **Vectorized calculations**

   * Compute `net_price = price * (1 - discount)` and `revenue = net_price * qty` (no loops).

3. **Filtering**

   * Keep only rows where `revenue > 10`. From these, show **top 5** by `revenue`.

4. **Grouping & aggregation**

   * Daily total revenue per `date`.
   * For each `(store, category)` pair, compute **total**, **mean**, and **count** of `revenue`.
   * Identify the highest-revenue **day** and best-performing **(store, category)**.

5. **Sanity checks**

   * Ensure **no NaNs** remain in `price`, `qty`, `net_price`, `revenue`.
   * Quick stats: `describe()` on `price`, `qty`, `revenue`.

6. **(Optional)** Save cleaned data to `clean_sales.csv`.

---

# Suggested Approach (at a glance)

1. **Generate data with NumPy**, then wrap it in a pandas DataFrame.
2. **Fill missing values** in one or two passes, using groupwise operations:

   * `groupby('category')['price'].ffill()`
   * `groupby('category')['price'].transform('median')` → `fillna`
   * `qty.fillna(1)`
3. **Vectorize** all numerical computations (avoid Python loops).
4. **Filter** with boolean masks; get top-k via `nlargest`.
5. **Aggregate** with `groupby(...).agg(...)`, sorting for insights.
6. **Validate** with `isna().sum()` and `describe()`; **save** if needed.

---


```

---



In [2]:
# Starter Code (fill the TODOs)


# --- Tiny Shop Analytics (Starter) ---
import numpy as np
import pandas as pd

# 0) Reproducible synthetic data
rng = np.random.default_rng(42)
n = 60  # number of transactions

dates = pd.date_range("2025-07-01", periods=n, freq="D")
store = rng.choice(["A", "B"], size=n)
category = rng.choice(["Food", "Home", "Toys"], size=n, p=[0.5, 0.3, 0.2])

price = rng.normal(20, 5, size=n).clip(1)     # >= 1
qty   = rng.poisson(3, size=n) + 1            # >= 1
discount = rng.choice([0.0, 0.1, 0.2], size=n, p=[0.6, 0.3, 0.1])

# Introduce ~5% missing values in price & qty
mask = rng.random(n) < 0.05
price[mask] = np.nan
mask = rng.random(n) < 0.05
# Change qty to float type to allow NaN values
qty = qty.astype(float)
qty[mask] = np.nan

df = pd.DataFrame({
    "date": dates,
    "store": store,
    "category": category,
    "price": price,
    "qty": qty,
    "discount": discount
})

print("Raw head:")
print(df.head(), "\n")

# 1) Handle missing values
#    - price: groupwise forward-fill within category, then category median for any remaining NaNs
#    - qty: fill NaN with 1
df['price'] = df.groupby('category')['price'].ffill()
med = df.groupby('category')['price'].transform('median')
df['price'] = df['price'].fillna(med)
df['qty'] = df['qty'].fillna(1)

# 2) Vectorized calculations (no loops)
#    - net_price = price * (1 - discount)
#    - revenue   = net_price * qty
df = df.assign(net_price = df['price'] * (1 - df['discount']))
df['revenue'] = df['net_price'] * df['qty']

# 3) Filtering and top-5 by revenue
#    - Keep only rows where revenue > 10 -> df_filt
#    - top5 = df_filt.nlargest(5, 'revenue')
df_filt = df[df['revenue'] > 10]
top5 = df_filt.nlargest(5, 'revenue')

# 4) Grouping & aggregation
#    - daily_rev: total revenue per date (sort by revenue desc)
#    - store_cat: for each (store, category), total/mean/count of revenue, sorted by total desc
daily_rev = (df.groupby('date', as_index=False)['revenue']
               .sum()
               .sort_values('revenue', ascending=False))
store_cat = (df.groupby(['store','category'], as_index=False)
               .agg(total=('revenue','sum'),
                    mean=('revenue','mean'),
                    transactions=('revenue','size'))
               .sort_values('total', ascending=False))


# 5) Sanity checks and quick stats
assert df[['price','qty','net_price','revenue']].isna().sum().sum() == 0
print(df[['price','qty','revenue']].describe())

# 6) (Optional) Save
# df.to_csv('clean_sales.csv', index=False)

# --- After you fill the TODOs, you might print some results like: ---
print("\nCleaned head:")
print(df.head())
print("\nTop 5 by revenue:")
print(top5)
print("\nDaily revenue (top 5 days):")
print(daily_rev.head())
print("\nStore x Category performance:")
print(store_cat)

Raw head:
        date store category      price  qty  discount
0 2025-07-01     A     Home  22.403733  5.0       0.0
1 2025-07-02     B     Toys  22.232656  6.0       0.1
2 2025-07-03     B     Food  23.326926  3.0       0.0
3 2025-07-04     A     Food  19.507573  5.0       0.0
4 2025-07-05     A     Food  17.883508  4.0       0.1 

           price        qty     revenue
count  60.000000  60.000000   60.000000
mean   19.489322   3.833333   69.627056
std     4.999690   1.606836   34.075680
min     9.339769   1.000000   11.563328
25%    15.446234   3.000000   47.268725
50%    19.134429   4.000000   58.810993
75%    22.187253   5.000000   89.591228
max    34.569312   8.000000  145.143797

Cleaned head:
        date store category      price  qty  discount  net_price     revenue
0 2025-07-01     A     Home  22.403733  5.0       0.0  22.403733  112.018666
1 2025-07-02     B     Toys  22.232656  6.0       0.1  20.009390  120.056342
2 2025-07-03     B     Food  23.326926  3.0       0.0  23.